In [1]:
import random
import httpx
import tempfile

# notebook parameters
PROFILE='demo'
EXAMPLE_ASSET_FOLDER=f'demo/test'
EXAMPLE_ASSET=f'{EXAMPLE_ASSET_FOLDER}/example_{random.randint(0,10000):04d}.txt'
EXAMPLE_ASSET
TEMP_DIR=tempfile.mkdtemp('storagedemo')

### Initialise the waylay client

In [33]:
from waylay import WaylayClient, __version__
__version__

'v0.7.0'

In [34]:
waylay = WaylayClient.from_profile(PROFILE)
waylay.config.to_dict()

{'credentials': {'type': 'client_credentials',
  'api_key': '41e9fa5124307d9ebeed7909',
  'api_secret': '********',
  'gateway_url': 'https://api.waylay.io',
  'accounts_url': None},
 'profile': 'demo',
 'settings': {'waylay_storage': 'https://storage.waylay.io'}}

# Using Waylay Storage in the python SDK


## Using `storage.content` to upload and download objects

The `content` utility calls to the Waylay Storage service to get signed URLs to the content, and then execute download or upload actions on these endpoints.

##### an example text file

In [35]:
file_name=f'{TEMP_DIR}/hi.txt'
with open(file_name, mode='w') as f:
    f.write('Hi storage demo')

##### put the file at the `EXAMPLE_ASSET` location

In [36]:
waylay.storage.content.put('assets', EXAMPLE_ASSET, from_file=file_name)

Uploading content to assets/demo/test/example_8363.txt ...
... done.


'OK'

##### get the file from the `EXAMPLE_ASSET` location

In [37]:
res= waylay.storage.content.get('assets', EXAMPLE_ASSET)

In [38]:
res.text

'Hi storage demo'

In [39]:
res.headers['content-type']

'binary/octet-stream'

##### use the correct content type (`text/plain`) to upload the file

In [40]:
waylay.storage.content.put('assets', EXAMPLE_ASSET, from_file=file_name, content_type='text/plain')

Uploading content to assets/demo/test/example_8363.txt ...
... done.


'OK'

In [41]:
res= waylay.storage.content.get('assets', EXAMPLE_ASSET)

In [42]:
res.headers['content-type']

'text/plain'

###### remove the object

In [50]:
waylay.storage.object.remove('assets', EXAMPLE_ASSET)

{'_links': {'removed': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt'}}}

### `storage.bucket` api
A _bucket_ is a collection of storage objects private to the Waylay tenant.

#### `storage.bucket.list`

In [51]:
f'You have access to {len(waylay.storage.bucket.list())} buckets on {waylay.config.auth.current_token.domain}'


'You have access to 5 buckets on demo.waylay.io'

#### `storage.bucket.get`

In [52]:
# the 'assets' bucket are generic objects that only waylay users of that tenant can access.
waylay.storage.bucket.get('assets')

{'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets'},
  'list': {'href': 'https://storage.waylay.io/bucket/assets/'},
  'sign_post': {'href': 'https://storage.waylay.io/bucket/assets/?sign=POST&expiry_seconds=300'},
  'subscriptions': {'href': 'https://storage.waylay.io/subscription/assets'},
  'store': {'href': 's3://prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21'}},
 'alias': 'assets',
 'name': 'prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21',
 'store': {'type': 'minio_s3',
  'name': 's3',
  'url': 'https://object-storage.waylay.io'}}

### `storage.object` api
A storage _object_ is a pointer to a single file.


#### `storage.object.list`

In [53]:
# list all object with a given prefix, gives an empty list when nothing is found.
waylay.storage.object.list('assets',EXAMPLE_ASSET_FOLDER)

[{'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/?stat=true'},
   'list': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/'},
   'sign_post': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/?sign=POST&expiry_seconds=300'},
   'sign_put': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/?sign=PUT&expiry_seconds=300'},
   'remove': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/',
    'method': 'DELETE'}},
  'bucket': {'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets'}},
   'alias': 'assets',
   'name': 'prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21'},
  'name': 'demo/test/',
  'etag': '',
  'size': 0,
  'is_dir': True}]

In [54]:
waylay.storage.object.list('assets',EXAMPLE_ASSET)

[]

#### `storage.object.stat`

In [55]:
# show metadata of a specific object, fail if not found
try:
    waylay.storage.object.stat('assets',EXAMPLE_ASSET)
except Exception as e:
    print(e)

RestResponseError(404: 'operation=not_found_error'; GET 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt?stat=true')


In [56]:
waylay.storage.content.put('assets',EXAMPLE_ASSET, content='Example')

Uploading content to assets/demo/test/example_8363.txt ...
... done.


'OK'

In [57]:
waylay.storage.object.stat('assets',EXAMPLE_ASSET)

{'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt?stat=true'},
  'sign_get': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt?sign=GET&expiry_seconds=300'},
  'sign_put': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt?sign=PUT&expiry_seconds=300'},
  'remove': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt',
   'method': 'DELETE'}},
 'bucket': {'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets'}},
  'alias': 'assets',
  'name': 'prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21'},
 'name': 'demo/test/example_8363.txt',
 'last_modified': '2023-10-16T12:02:58+00:00',
 'etag': '0a52730597fb4ffa01fc117d9e71e3a9',
 'size': 7,
 'content_type': 'binary/octet-stream',
 'is_dir': False}

In [58]:
res = waylay.storage.content.get('assets',EXAMPLE_ASSET)
res.text

'Example'

In [59]:
waylay.storage.object.list('assets','', params={ 'max_keys':1})

[{'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets/50MBFile?stat=true'},
   'sign_get': {'href': 'https://storage.waylay.io/bucket/assets/50MBFile?sign=GET&expiry_seconds=300'},
   'sign_put': {'href': 'https://storage.waylay.io/bucket/assets/50MBFile?sign=PUT&expiry_seconds=300'},
   'remove': {'href': 'https://storage.waylay.io/bucket/assets/50MBFile',
    'method': 'DELETE'}},
  'bucket': {'_links': {'self': {'href': 'https://storage.waylay.io/bucket/assets'}},
   'alias': 'assets',
   'name': 'prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21'},
  'name': '50MBFile',
  'last_modified': '2023-09-20T17:09:29+00:00',
  'etag': '25e317773f308e446cc84c503a6d1f85',
  'size': 52428800,
  'is_dir': False}]

#### `storage.object.iter_list_all`

In [60]:
# iterate over all items using a paging generator
# note that this can invoke multiple http requests if the list is huge
count=0
for x in waylay.storage.object.iter_list_all('assets','', params={'recursive': True}):
  print(x['name'])
  count += 1
  if count >= 5:
    break

50MBFile
demo/test/example_6402.csv
demo/test/example_8363.txt
demo/test/example_9064.txt
demo/test/example_9087.txt


## Advanced usage: signed urls

#### `storage.object.sign_get` ,  `storage.object.sign_put`,   `storage.object.sign_post` 

In [61]:
### SEE `storage.content` API above for shortcuts ...
# create a signed url for uploading to a (existing or new) location
put_url = waylay.storage.object.sign_put('assets',EXAMPLE_ASSET)
httpx.put(put_url, content='Hello Waylay Demo')
# creat a signed url to read a location
get_url = waylay.storage.object.sign_get('assets',EXAMPLE_ASSET)
httpx.get(get_url).text

'Hello Waylay Demo'

In [62]:
# an example file
file_name=f'{TEMP_DIR}/hi.txt'
with open(file_name, mode='w') as f:
    f.write('Hi storage demo')
    
with open(file_name, mode='r') as f:
    print(f.read())

Hi storage demo


In [63]:
## use these POST parameters to post an object to any path below the signed pat
post_cfg=waylay.storage.object.sign_post('assets',EXAMPLE_ASSET_FOLDER, params={'content-type':'text/plain'})
post_cfg

{'href': 'https://object-storage.waylay.io/prod-ws-assets-fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21/',
 'method': 'POST',
 'form_data': {'x-amz-algorithm': 'AWS4-HMAC-SHA256',
  'x-amz-credential': 'fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21.assets_readwritedelete/20231016/eu-west-1/s3/aws4_request',
  'x-amz-date': '20231016T120334Z',
  'policy': 'eyJleHBpcmF0aW9uIjogIjIwMjMtMTAtMTZUMTI6MDg6MzQuMjE2WiIsICJjb25kaXRpb25zIjogW1siZXEiLCAiJGJ1Y2tldCIsICJwcm9kLXdzLWFzc2V0cy1mYTMxZWM1Zi1iZjZkLTRjZGItOTMwZi0yY2RkMzhiNTNlMjEiXSwgWyJzdGFydHMtd2l0aCIsICIka2V5IiwgImRlbW8vdGVzdCJdLCBbImVxIiwgIiR4LWFtei1hbGdvcml0aG0iLCAiQVdTNC1ITUFDLVNIQTI1NiJdLCBbImVxIiwgIiR4LWFtei1jcmVkZW50aWFsIiwgImZhMzFlYzVmLWJmNmQtNGNkYi05MzBmLTJjZGQzOGI1M2UyMS5hc3NldHNfcmVhZHdyaXRlZGVsZXRlLzIwMjMxMDE2L2V1LXdlc3QtMS9zMy9hd3M0X3JlcXVlc3QiXSwgWyJlcSIsICIkeC1hbXotZGF0ZSIsICIyMDIzMTAxNlQxMjAzMzRaIl1dfQ==',
  'x-amz-signature': '245c2623325a8a8fc6092e88b6669f7c5280f961ba3b25c6f70854bfa58e7eb6',
  'bucket': 'prod-ws-assets-fa31ec5f-bf6d-4cdb-9

In [64]:
# use any http library that supports form data:
# for httpx: https://www.python-httpx.org/quickstart/#sending-multipart-file-uploads
with open(file_name, mode='rb') as f: # needs a binary stream
    res = httpx.post(
        post_cfg['href'], # the object-storage url
        files={ 'file': f }, 
        data={ 
            **post_cfg['form_data'], # copy over the signed form data
            'key': EXAMPLE_ASSET # override 'key' to the exact path (must have prefix compatible with the signed url)
        }
    )
res

<Response [204 No Content]>

In [65]:
waylay.storage.content.get('assets', EXAMPLE_ASSET).text

'Hi storage demo'

In [66]:
waylay.storage.object.remove('assets', EXAMPLE_ASSET)

{'_links': {'removed': {'href': 'https://storage.waylay.io/bucket/assets/demo/test/example_8363.txt'}}}